Dependencies

In [ ]:
!pip install tensorflow
!pip install Keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.backend import set_session
import collections
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import os
from keras.models import load_model
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras import losses
from keras.optimizers import SGD
from keras.layers import RepeatVector

Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = open('/content/drive/MyDrive/Tamil.txt', 'r+', encoding="utf8")
x = f.readlines()

In [ ]:
f = open('/content/drive/MyDrive/English.txt', 'r+', encoding="utf8")
y = f.readlines()

In [ ]:
x = x[0:1000]
len(x)

1000

In [ ]:
y = y[0:1000]

Text Preprocessing

In [ ]:
x[0]= x[0].strip('\ufeffMMA')
y[0]= y[0].strip('\ufeffMMA')

In [ ]:
import string
exclude = set(string.punctuation)
for i in range(0,len(x)):
    x[i] = x[i].strip('\n')
    x[i] = ''.join(ch for ch in x[i] if ch not in exclude)

In [ ]:
for i in range(0,len(y)):
    y[i] = y[i].lower()
    y[i] = y[i].strip('\n')
    y[i] = ''.join(ch for ch in y[i] if ch not in exclude)

Cleaned text

In [ ]:
print("Tamil Text:",x[1],"\n")
print("English Text:",y[1])

Tamil Text: சமீபகாலத்தில் சில தகவல்கள் யூலியஸ் ரோசன்பேர்க் ஒரு வித உளவுச்செய்தியை சோவியத் அதிகாரிகளுக்கு இரண்டாம் உலகப்போரின்போது அனுப்பியதில் சம்பந்தப்பட்டு இருந்ததாக வெளிவந்துள்ளன 

English Text: information has surfaced in recent years suggesting that julius rosenberg was involved in passing some form of intelligence to soviet officials during the second world war


In [ ]:
len(x)

1000

In [ ]:
len(y)

1000

In [ ]:
english_words = []
for i in range(0,len(y)):
    english_words.append(y[i].split())   

In [ ]:
english_words = [j for sub in english_words for j in sub]

In [ ]:
print("Number of Unique English words:",len(set(english_words)))

Number of Unique English words: 5632


In [ ]:
tamil_words = []
for i in range(0,len(x)):
    tamil_words.append(x[i].split())  
tamil_words = [j for sub in tamil_words for j in sub]

In [ ]:
print("Number of Unique Tamil words:",len(set(tamil_words)))

Number of Unique Tamil words: 9331


In [ ]:
tamilvocab = len(set(tamil_words))
engvocab = len(set(english_words))

In [ ]:
length_tamil=[]
for i in range(0,len(x)):
    length_tamil.append(len(x[i].split()))

In [ ]:
length_english=[]
for i in range(0,len(y)):
    length_english.append(len(y[i].split()))

In [ ]:
sum(length_english)/len(length_english)

22.953

In [ ]:
sum(length_tamil)/len(length_tamil)

15.999

In [ ]:
print(max(length_english))
print(max(length_tamil))

112
76


In [ ]:
english_words_counter = collections.Counter([word for sentence in y for word in sentence.split()])
tamil_words_counter = collections.Counter([word for sentence in x for word in sentence.split()])

In [ ]:
english_words_counter.most_common(10)

[('the', 1881),
 ('of', 892),
 ('and', 801),
 ('to', 656),
 ('in', 524),
 ('a', 395),
 ('that', 294),
 ('is', 234),
 ('for', 222),
 ('on', 168)]

In [ ]:
tamil_words_counter.most_common(10)

[('ஒரு', 181),
 ('மற்றும்', 168),
 ('என்று', 123),
 ('இந்த', 96),
 ('அவர்', 65),
 ('அமெரிக்க', 51),
 ('என்ற', 47),
 ('என', 43),
 ('நான்', 42),
 ('அரசியல்', 39)]

Tokenizer

In [ ]:
!pip install -U protobuf==3.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
# config = tf.compat.v1.ConfigProto
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# set_session(tf.Session(config=config))

In [ ]:
from keras.preprocessing.text import Tokenizer
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x) 
    return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
x[0]

' கட்சியின் துணைத்தலைவர் க்வாஸி ஹுசேன் அகமத் சென்ற மாதம் பின்வருமாறு அறிவித்தார் நாங்கள் தீவிரவாதிகள் அல்ல'

In [ ]:
z=(tokenize(x))
z[0][1]

[1970,
 61,
 964,
 1971,
 1972,
 1,
 1973,
 1974,
 303,
 593,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980]

In [ ]:
z[1]

In [ ]:
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',]
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'brown': 3, 'fox': 4, 'jumps': 5, 'over': 6, 'lazy': 7, 'dog': 8, 'by': 9, 'jove': 10, 'my': 11, 'study': 12, 'of': 13, 'lexicography': 14, 'won': 15, 'a': 16, 'prize': 17}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 3, 4, 5, 6, 1, 7, 8]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [9, 10, 11, 2, 12, 13, 14, 15, 16, 17]


Padding

In [ ]:
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

In [ ]:
test_pad = pad(text_tokenized)
print("OUTPUT IS ALWAYS A LENGTH 10 ARRAY....FILLED BY 0s IN THE END")
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

OUTPUT IS ALWAYS A LENGTH 10 ARRAY....FILLED BY 0s IN THE END
Sequence 1 in x
  Input:  [1 2 3 4 5 6 1 7 8]
  Output: [1 2 3 4 5 6 1 7 8 0]
Sequence 2 in x
  Input:  [ 9 10 11  2 12 13 14 15 16 17]
  Output: [ 9 10 11  2 12 13 14 15 16 17]


Applying all tested preprocessing functions to our corpus

In [ ]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    print('shape before: ', preprocess_y.shape)
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    print('shape after: ', preprocess_y.shape)
        
    return preprocess_x, preprocess_y, x_tk, y_tk

In [ ]:
preprocess_x, x_tk = tokenize(x)
preprocess_x = pad(preprocess_x)
preprocess_x.shape

(1000, 76)

In [ ]:
preproc_tamil_sentences, preproc_english_sentences, tamil_tokenizer, english_tokenizer =\
    preprocess(x, y)

shape before:  (1000, 112)
shape after:  (1000, 112, 1)


In [ ]:
list(tamil_tokenizer.word_index.items())[:5]

[('ஒரு', 1), ('மற்றும்', 2), ('என்று', 3), ('இந்த', 4), ('அவர்', 5)]

In [ ]:
list(english_tokenizer.word_index.items())[:5]

[('the', 1), ('of', 2), ('and', 3), ('to', 4), ('in', 5)]

Logits to text

In [ ]:
def logits_to_text(logits, tokenizer):

    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
print("tamil_sentences shape: ", preproc_english_sentences.shape)
print("english_sentences  shape: ", preproc_tamil_sentences.shape)
print('output sequence length: ', preproc_english_sentences.shape[1])

tamil_sentences shape:  (1000, 112, 1)
english_sentences  shape:  (1000, 76)
output sequence length:  112


In [ ]:
tmp_x = pad(preproc_tamil_sentences, preproc_english_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2], 1))

In [ ]:
tmp_x.shape

(1000, 112, 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test , y_train , y_test = train_test_split(
...     tmp_x, preproc_english_sentences, test_size=0.2, random_state=42)

In [ ]:
def simple_model(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size, learning_rate=0.1):
    model = Sequential()
    model.add(GRU(128, dropout=0.2,input_shape=input_shape[1:], return_sequences=True) )
    model.add(TimeDistributed(Dense(english_vocab_size, activation='Softmax') ))
    print('######## Summary ###########')
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [ ]:
simple_rnn_model = simple_model(
    x_train.shape,
    preproc_english_sentences.shape[1],
    engvocab+1,
    tamilvocab+1)

###########LOAD MODEL IF IT EXISTS OR ELSE TRAIN THE MODEL################

if os.path.exists(os.path.join("model", "translation.h5"))== False:
    rnn = simple_rnn_model.fit(x_train, y_train, batch_size=150, epochs=50)
else:
    simple_rnn_model = load_model(os.path.join("model", "translation.h5"))

######## Summary ###########
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_7 (GRU)                 (None, 112, 128)          50304     
                                                                 
 time_distributed_7 (TimeDis  (None, 112, 9332)        1203828   
 tributed)                                                       
                                                                 
Total params: 1,254,132
Trainable params: 1,254,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
6/6 [==============================] - 42s 6s/step - loss: 5.2376 - accuracy: 0.7466
Epoch 2/50
6/6 [==============================] - 37s 6s/step - loss: 2.5195 - accuracy: 0.7985
Epoch 3/50
6/6 [==============================] - 39s 6s/step - loss: 2.4534 - accuracy: 0.7650
Epoch 4/50
6/6 [==============================] - 39s 6s/

Save Model

In [ ]:
simple_rnn_model.save(os.path.join("model", "translation.h5"))

In [ ]:
train_score = simple_rnn_model.evaluate(x_train, y_train, verbose=0)
print(" Train Accuracy: ", train_score[1])

 Train Accuracy:  0.8066071271896362


In [ ]:
test_score = simple_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.803928554058075


In [ ]:
def simple_model(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size, learning_rate=0.1):
    model = Sequential()
    model.add(GRU(128, dropout=0.3,input_shape=input_shape[1:], return_sequences=True) )
    model.add(TimeDistributed(Dense(english_vocab_size, activation='Softmax') ))
    print('######## Summary ###########')
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [ ]:
simple_rnn_model = simple_model(
    x_train.shape,
    preproc_english_sentences.shape[1],
    engvocab+1,
    tamilvocab+1)

###########LOAD MODEL IF IT EXISTS OR ELSE TRAIN THE MODEL################

if os.path.exists(os.path.join("model", "translation.h5"))== False:
    rnn = simple_rnn_model.fit(x_train, y_train, batch_size=200, epochs=50)
else:
    simple_rnn_model = load_model(os.path.join("model", "translation.h5"))

######## Summary ###########
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_9 (GRU)                 (None, 112, 128)          50304     
                                                                 
 time_distributed_9 (TimeDis  (None, 112, 9332)        1203828   
 tributed)                                                       
                                                                 
Total params: 1,254,132
Trainable params: 1,254,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
4/4 [==============================] - 43s 9s/step - loss: 6.3658 - accuracy: 0.3592
Epoch 2/50
4/4 [==============================] - 38s 10s/step - loss: 2.6069 - accuracy: 0.8000
Epoch 3/50
4/4 [==============================] - 39s 10s/step - loss: 2.5026 - accuracy: 0.7988
Epoch 4/50
4/4 [==============================] - 36s 9

In [ ]:
test_score = simple_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.8031696677207947


In [ ]:
def simple_model(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size, learning_rate=0.1):
    model = Sequential()
    model.add(GRU(128, dropout=0.1,input_shape=input_shape[1:], return_sequences=True) )
    model.add(TimeDistributed(Dense(english_vocab_size, activation='Softmax') ))
    print('######## Summary ###########')
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [ ]:
simple_rnn_model = simple_model(
    x_train.shape,
    preproc_english_sentences.shape[1],
    engvocab+1,
    tamilvocab+1)

###########LOAD MODEL IF IT EXISTS OR ELSE TRAIN THE MODEL################

if os.path.exists(os.path.join("model", "translation.h5"))== False:
    rnn = simple_rnn_model.fit(x_train, y_train, batch_size=50, epochs=50)
else:
    simple_rnn_model = load_model(os.path.join("model", "translation.h5"))

######## Summary ###########
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 112, 128)          50304     
                                                                 
 time_distributed (TimeDistr  (None, 112, 9332)        1203828   
 ibuted)                                                         
                                                                 
Total params: 1,254,132
Trainable params: 1,254,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
16/16 [==============================] - 48s 3s/step - loss: 3.4382 - accuracy: 0.6805
Epoch 2/50
16/16 [==============================] - 42s 3s/step - loss: 2.2270 - accuracy: 0.8015
Epoch 3/50
16/16 [==============================] - 42s 3s/step - loss: 2.1084 - accuracy: 0.8025
Epoch 4/50
16/16 [==============================] - 4

In [ ]:
train_score = simple_rnn_model.evaluate(x_train, y_train, verbose=0)
print(" Train Accuracy: ", train_score[1])

 Train Accuracy:  0.8059263229370117


In [ ]:
test_score = simple_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.803303599357605
